In [1]:
%load_ext sql

In [2]:
%sql sqlite://

'Connected: @None'

# Chicago Socioeconomic Data

In [3]:
import pandas
chicago_socioeconomic_data = pandas.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012-v3.csv')
%sql PERSIST chicago_socioeconomic_data

 * sqlite://


'Persisted chicago_socioeconomic_data'

In [4]:
%sql SELECT * FROM chicago_socioeconomic_data LIMIT 5;

 * sqlite://
Done.


index,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [5]:
%sql PRAGMA table_info(chicago_socioeconomic_data)

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,index,BIGINT,0,None,0
1,COMMUNITY_AREA_NUMBER,BIGINT,0,None,0
2,COMMUNITY_AREA_NAME,TEXT,0,None,0
3,PERCENT OF HOUSING CROWDED,FLOAT,0,None,0
4,PERCENT HOUSEHOLDS BELOW POVERTY,FLOAT,0,None,0
5,PERCENT AGED 16+ UNEMPLOYED,FLOAT,0,None,0
6,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,TEXT,0,None,0
7,PERCENT AGED UNDER 18 OR OVER 64,FLOAT,0,None,0
8,PER_CAPITA_INCOME,BIGINT,0,None,0
9,HARDSHIP_INDEX,TEXT,0,None,0


As we can see **PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA** and **HARDSHIP_INDEX** supposed to be a FLOAT. Howeverm they are defined as a TEXT here. Let's check to find out the reasons. 

In [6]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, `PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA`, HARDSHIP_INDEX,
CASE   
    WHEN CAST(`PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA` AS INTEGER) THEN 'Number'
    ELSE 'Character'
END AS data_type1,
CASE   
    WHEN CAST(HARDSHIP_INDEX AS INTEGER) THEN 'Number'
    ELSE 'Character'
END AS data_type2
from chicago_socioeconomic_data 
WHERE data_type1 = "Character" or data_type2 = "Character"

 * sqlite://
Done.


COMMUNITY_AREA_NUMBER,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,HARDSHIP_INDEX,data_type1,data_type2
8,2.5,A1,Number,Character
12,< 4.9,11,Character,Number
15,Around 19.3%,35,Character,Number
20,41.6,BCD71,Number,Character


There are 4 problems here:
- **COMMUNITY_AREA_NUMBER** 8 **HARDSHIP_INDEX** should be changed from A1 to 1
- **COMMUNITY_AREA_NUMBER** 12 **PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA** should be changed from < 4.9 to 4.9
- **COMMUNITY_AREA_NUMBER** 12 **PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA** should be changed from Around 19.3% to 19.3
- **COMMUNITY_AREA_NUMBER** 8 **HARDSHIP_INDEX** should be changed from BCD71 to 71

In [7]:
%%sql
UPDATE chicago_socioeconomic_data SET
HARDSHIP_INDEX = 
    CASE 
        WHEN COMMUNITY_AREA_NUMBER = 8 THEN 1 
        WHEN COMMUNITY_AREA_NUMBER = 20 THEN 71 
        ELSE HARDSHIP_INDEX END

 * sqlite://
77 rows affected.


[]

In [8]:
%%sql
UPDATE chicago_socioeconomic_data SET
`PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA` = 
    CASE 
        WHEN COMMUNITY_AREA_NUMBER = 12 THEN 4.9 
        WHEN COMMUNITY_AREA_NUMBER = 15 THEN 19.3 
        ELSE `PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA` END

 * sqlite://
77 rows affected.


[]

Let's check to see whether these 4 rows have been corrected

In [9]:
%%sql 
SELECT COMMUNITY_AREA_NUMBER, `PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA`, HARDSHIP_INDEX 
FROM chicago_socioeconomic_data
WHERE COMMUNITY_AREA_NUMBER IN (8,12,15,20)

 * sqlite://
Done.


COMMUNITY_AREA_NUMBER,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,HARDSHIP_INDEX
8,2.5,1
12,4.9,11
15,19.3,35
20,41.6,71


We are going to check the MAX and MIN of every numeric column to see whether there is any weird values.

In [10]:
%%sql
SELECT MAX(`PERCENT OF HOUSING CROWDED`) AS `PERCENT OF HOUSING CROWDED`, 
       MAX(`PERCENT HOUSEHOLDS BELOW POVERTY`) AS `PERCENT HOUSEHOLDS BELOW POVERTY`, 
       MAX(`PERCENT AGED 16+ UNEMPLOYED`) AS `PERCENT AGED 16+ UNEMPLOYED`, 
       MAX(`PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA`) AS `PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA`,
       MAX(`PERCENT AGED UNDER 18 OR OVER 64`) AS `PERCENT AGED UNDER 18 OR OVER 64`, 
       MAX(`PER_CAPITA_INCOME `) AS `PER_CAPITA_INCOME `, 
       MAX(HARDSHIP_INDEX) AS HARDSHIP_INDEX
FROM chicago_socioeconomic_data
UNION ALL
SELECT MIN(`PERCENT OF HOUSING CROWDED`), MIN(`PERCENT HOUSEHOLDS BELOW POVERTY`), 
       MIN(`PERCENT AGED 16+ UNEMPLOYED`), MIN(`PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA`),
       MIN(`PERCENT AGED UNDER 18 OR OVER 64`), MIN(`PER_CAPITA_INCOME `), MIN(HARDSHIP_INDEX)
FROM chicago_socioeconomic_data

 * sqlite://
Done.


PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER_CAPITA_INCOME,HARDSHIP_INDEX
15.8,56.5,100.0,9.7,51.5,1093400000,98
0.3,-42.4,4.7,10.6,13.5,8201,1


There are definitely some weird values in here. 
- **PERCENT HOUSEHOLDS BELOW POVERTY** cannot be negative, probably it is 42.4 instead of -42.4. To make sure there is no other negative values, I am going to check this column.
- **PERCENT AGED 16+ UNEMPLOYED** is very unlikely to be 100, probably it is 10
- **PER_CAPITA_INCOME** max value is 1093400000 which is impossible, most likely it is 10934. In addition, from my knowledge, **PER_CAPITA_INCOME** is certainly lower than 100,000. I will take a look once again at this column to ensure all the data is correct.	

In [11]:
%%sql
SELECT * FROM chicago_socioeconomic_data
WHERE `PERCENT HOUSEHOLDS BELOW POVERTY` <= 0

 * sqlite://
Done.


index,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER_CAPITA_INCOME,HARDSHIP_INDEX
13,14,Albany Park,11.3,-19.2,10.0,32.9,32.0,21323,53
26,27,East Garfield Park,8.2,-42.4,19.6,21.3,43.2,12961,83


There is a row another row with negative value. It is most likely to be 19.2 instead of -19.2

In [12]:
%%sql
SELECT * FROM chicago_socioeconomic_data
WHERE `PER_CAPITA_INCOME ` > 100000

 * sqlite://
Done.


index,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER_CAPITA_INCOME,HARDSHIP_INDEX
15,16,Irving Park,6.3,13.1,10.0,22.4,31.6,27249000,34
25,26,West Garfield Park,9.4,41.7,25.8,24.5,43.6,1093400000,92


There is another row with wrong **PER_CAPITA_INCOME** value, it should be 27249 instead of 27249000. We are going to fix all these errors before getting insights.	

In [13]:
%%sql
UPDATE chicago_socioeconomic_data SET
`PERCENT HOUSEHOLDS BELOW POVERTY` = 
    CASE 
        WHEN `PERCENT HOUSEHOLDS BELOW POVERTY` = -42.4 THEN 42.4
        WHEN `PERCENT HOUSEHOLDS BELOW POVERTY` = -19.2 THEN 19.2
        ELSE `PERCENT HOUSEHOLDS BELOW POVERTY` END

 * sqlite://
77 rows affected.


[]

In [14]:
%%sql
UPDATE chicago_socioeconomic_data SET
`PERCENT AGED 16+ UNEMPLOYED` = 
    CASE 
        WHEN `PERCENT AGED 16+ UNEMPLOYED` = 100 THEN 10
        ELSE `PERCENT AGED 16+ UNEMPLOYED` END

 * sqlite://
77 rows affected.


[]

In [15]:
%%sql
UPDATE chicago_socioeconomic_data SET
`PER_CAPITA_INCOME ` =
    CASE
        WHEN `PER_CAPITA_INCOME ` = 27249000 THEN 27249
        WHEN `PER_CAPITA_INCOME ` = 1093400000 THEN 10934
        ELSE `PER_CAPITA_INCOME ` END

 * sqlite://
77 rows affected.


[]

### 1. How many rows in the dataset ?

In [16]:
%%sql 
SELECT COUNT(*) AS no_of_rows FROM chicago_socioeconomic_data ; 

 * sqlite://
Done.


no_of_rows
77


### 2. How many community areas in Chicago have a hardship index greater than 50.0?

In [17]:
%%sql
SELECT COUNT(community_area_name) FROM chicago_socioeconomic_data
WHERE hardship_index > 50;

 * sqlite://
Done.


COUNT(community_area_name)
41


### 3. What is the maximum value of hardship index in this dataset?

In [18]:
%%sql
SELECT MAX(hardship_index) FROM chicago_socioeconomic_data;

 * sqlite://
Done.


MAX(hardship_index)
98


### 4. Which community area which has the highest percent households below poverty ? 

In [19]:
%%sql
SELECT community_area_name FROM chicago_socioeconomic_data
WHERE `PERCENT HOUSEHOLDS BELOW POVERTY` = 
(SELECT MAX(`PERCENT HOUSEHOLDS BELOW POVERTY`) FROM chicago_socioeconomic_data)

 * sqlite://
Done.


COMMUNITY_AREA_NAME
Riverdale


### 5. Which community area has "Park" in its name ?

In [20]:
%%sql
SELECT community_area_name FROM chicago_socioeconomic_data
WHERE community_area_name LIKE '%Park%'

 * sqlite://
Done.


COMMUNITY_AREA_NAME
Rogers Park
Lincoln Park
Edison Park
Norwood Park
Jefferson Park
North Park
Albany Park
Portage Park
Irving Park
Humboldt park


# Chicago Crime Data

In [21]:
chicago_crime_data = pandas.read_csv('Chicago_Crime_Data-v2.csv')
%sql PERSIST chicago_crime_data

 * sqlite://


'Persisted chicago_crime_data'

In [22]:
%sql SELECT * FROM chicago_crime_data LIMIT 5;

 * sqlite://
Done.


index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807440500000006,-87.70395585,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,0,0,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,0,0,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455299999995,-87.70648047,"(41.773455295, -87.706480471)"


### 1. Find the total number of crimes recorded in the crime table.

In [23]:
%%sql
SELECT COUNT(*) AS total FROM chicago_crime_data;

 * sqlite://
Done.


total
533


### 2. How many crimes involve an arrest. 

In [24]:
%%sql
SELECT COUNT(ID) FROM chicago_crime_data
WHERE arrest = 1;

 * sqlite://
Done.


COUNT(ID)
163


### 3. What  are the unique types of crimes ?

In [65]:
%%sql
SELECT DISTINCT(primary_type) FROM chicago_crime_data

 * sqlite://
Done.


PRIMARY_TYPE
THEFT
BATTERY
CRIMINAL DAMAGE
NARCOTICS
OTHER OFFENSE
ASSAULT
BURGLARY
MOTOR VEHICLE THEFT
DECEPTIVE PRACTICE
ROBBERY


### 4. Which unique types of crimes (e.g. THEFT) have been recorded at a GAS STATION locations? 

In [29]:
%%sql
SELECT DISTINCT(primary_type) FROM chicago_crime_data
WHERE location_description LIKE "%Gas%"

 * sqlite://
Done.


PRIMARY_TYPE
THEFT
NARCOTICS
ROBBERY
CRIMINAL TRESPASS


### 5. How many crimes in each community area (with name) in ascending order ?

In [55]:
%%sql
SELECT community_area_name, COUNT(primary_type) AS no_of_crimes
FROM chicago_crime_data LEFT JOIN chicago_socioeconomic_data
ON chicago_socioeconomic_data.COMMUNITY_AREA_NUMBER = chicago_crime_data.COMMUNITY_AREA_NUMBER 
GROUP BY community_area_name
ORDER BY no_of_crimes ASC

 * sqlite://
Done.


COMMUNITY_AREA_NAME,no_of_crimes
Bridgeport,1
Burnside,1
Forest Glen,1
Hegewisch,1
Morgan Park,1
Near South Side,1
Clearing,2
East Side,2
Edgewater,2
Fuller Park,2


### 6. Which type of crime appears the most in each community area ? 

In [61]:
%%sql
SELECT community_area_name, primary_type, MAX(no_of_crimes) FROM
(SELECT community_area_name, primary_type, COUNT(primary_type) AS no_of_crimes
FROM chicago_crime_data LEFT JOIN chicago_socioeconomic_data
ON chicago_socioeconomic_data.COMMUNITY_AREA_NUMBER = chicago_crime_data.COMMUNITY_AREA_NUMBER 
GROUP BY community_area_name, primary_type)
GROUP BY community_area_name

 * sqlite://
Done.


community_area_name,primary_type,MAX(no_of_crimes)
None,BATTERY,11
Albany Park,THEFT,3
Ashburn,CRIMINAL DAMAGE,2
Auburn Gresham,ASSAULT,2
Austin,BATTERY,10
Avalon Park,ASSAULT,1
Avondale,BATTERY,2
Belmont Cragin,BATTERY,3
Beverly,BURGLARY,2
Bridgeport,CRIMINAL DAMAGE,1


### 7. Has the number of crime decrease  since 2011?

In [67]:
%%sql
SELECT year, COUNT(primary_type) FROM chicago_crime_data
GROUP BY year
HAVING year >= 2011

 * sqlite://
Done.


YEAR,COUNT(primary_type)
2011,25
2012,30
2013,24
2014,25
2015,23
2016,26
2017,30
2018,3
